<a href="https://colab.research.google.com/github/NicoPatalagua/BigData/blob/master/Spark_Dataframes_Windows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Taller Spark-Dataframes-Windows**
## *Paula Oviedo - Nicolás Patalagua*
## Infraestructura para BigData

# **Spark**
[Apache Spark](https://spark.apache.org/) es un framework de computación en clúster open-source. Spark fue desarrollado en sus inicios por Matei Zaharia en la Universidad de California,el AMPLab de la UC Berkeley en 2009. Fue liberado como código abierto en 2010 bajo licencia BSD. El código base del proyecto Spark fue donado más tarde a la Apache Software Foundation que se encarga de su mantenimiento desde entonces. 

## **¿Para que sirve Spark?**

*Spark proporciona una interfaz para la programación de clusters completos con Paralelismo de Datos implícito y tolerancia a fallos.Apache Spark se puede considerar un sistema de computación en clúster de propósito general y orientado a la velocidad. Proporciona APIs en Java, Scala, Python y R. También proporciona un motor optimizado que soporta la ejecución de grafos en general. También soporta un conjunto extenso y rico de herramientas de alto nivel entre las que se incluyen Spark SQL (para el procesamiento de datos estructurados basada en SQL), MLlib para implementar machine learning, GraphX para el procesamiento de grafos y Spark Streaming.*

# **Funciones de ventana en Spark**
*Las funciones de ventana permiten a los usuarios de Spark SQL calcular resultados como el rango de una fila determinada o un promedio móvil en un rango de filas de entrada. Mejoran significativamente la expresividad de las API de SQL y DataFrame de Spark.
En esencia, una función de ventana calcula un valor de retorno para cada fila de entrada de una tabla en función de un grupo de filas, llamado Marco . Cada fila de entrada puede tener un marco único asociado. Esta característica de las funciones de ventana las hace más potentes que otras funciones y permite a los usuarios expresar varias tareas de procesamiento de datos que son difíciles (si no imposibles) de expresar de manera concisa sin las funciones de ventana.* 

http://queirozf.com/entries/spark-dataframe-examples-window-functions

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark
import os
import time 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

from google.colab import drive
drive.mount('/content/gdrive')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("app")
sc = SparkContext.getOrCreate();

     |████████████████████████████████| 217.8MB 54kB/s 
     |████████████████████████████████| 204kB 44.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=d8274614e957c8b4058b5efe5a7c3597c869494bc5f62f137956c380d51ce65e
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# ***Taller***
**1. Descargar las acciones de Ecopetrol, Avianca, y Grupo Aval para el último año de Yahoo finances(23 de sept-2018 y 23 de sept-2019).** 

*Los CSV de acciones se obtienen de [Yahho! Finance](https://finance.yahoo.com/), el cual es un servicio de Yahoo! que proporciona información financiera y comentarios con un enfoque en los mercados de los Estados Unidos.*



Las acciones de [Ecopetrol](www.ecopetrol.com.co) se pueden consultar en [EC-Yahoo! Finance](https://finance.yahoo.com/quote/EC?p=EC&.tsrc=fin-srch), y pertenecen a la primera compañía de petróleo de Colombia, antiguamente llamada Empresa Colombiana de Petróleos S.A.Es la cuarta empresa petrolera más grande de Latinoamérica.

Las acciones de [Avianca S.A](www.avianca.com) se pueden consultar en [AVH-Yahoo! Finance](https://finance.yahoo.com/quote/AVH?p=AVH&.tsrc=fin-srch), y pertenecen a Aerovías del Continente Americano, anteriormente Aerovías Nacionales de Colombia. Es la mayor aerolínea de Colombia. Fundada en 1919 con el nombre de SCADTA, es mundialmente la segunda aerolínea más antigua después de la KLM que fue fundada 58 días antes y la más antigua del mundo con operaciones ininterrumpidas.

Las acciones del [Grupo Aval](www.grupoaval.com) se pueden consultar en [AVA-Yahoo! Finance](https://finance.yahoo.com/quote/AVAL?p=AVAL&.tsrc=fin-srch), corresponden a las acciones del Grupo Acciones y Valores S.A, un n conglomerado empresarial colombiano dedicado a una amplia variedad de actividades, principalmente financieras. El Grupo AVAL es controlado por Luis Carlos Sarmiento Angulo.

Una vez descargados son agregados a un repositorio de Github:


*   [EC](https://github.com/NicoPatalagua/BigData/blob/master/EC.csv)
*   [AVH](https://github.com/NicoPatalagua/BigData/blob/master/AVH.csv)
*   [AVA](https://github.com/NicoPatalagua/BigData/blob/master/AVA.csv)



**2.Porcentaje de retorno diaro.**


**3.Porcentaje de retorno Mensual.**


**4.Mes con mayor porcentaje de retorno.**


**5.Hallar y gráficar la media móvil de 20 dias para Ecopetrol.**


**6.Graficar comportamiento del precio de cierre para las 3 acciones a lo largo del año.**


**7.Calcular y graficar franajas Bollinger para K=2.**


**8.Para qué fechas la tendecia se sale de las franjas calculadas en el punto anterior?**

**9.Calcular y graficar las franjas Bollinger con K=2 y K=1.**

**10.Determinar para qué fechas es conveniente comprar usando la técnica de doble franja Bollinger.**

**11.Determinar para qué fechas es conveniente vender en corto la técnica de doble franja Bollinger.**